In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [2]:
df = pd.read_csv("./data/CaliforniaHousing/cal_housing.data", header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
X = df.iloc[:,:-1].values
y = np.log(df.iloc[:,-1].values)

### Train, test, scaler

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.datasets import fetch_california_housing


In [7]:
housing = fetch_california_housing()

In [8]:
housing.data[0]

array([   8.3252    ,   41.        ,    6.98412698,    1.02380952,
        322.        ,    2.55555556,   37.88      , -122.23      ])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, random_state=42)
scl = StandardScaler()
X_train_scl = scl.fit_transform(X_train)
X_test_scl = scl.transform(X_test)

### Custom training loop

We will implement a custom training loop:

- A way to generate batches from where to sample.
- For every epoch:
 - for every step in an epoch:
     - do a gradient update (update params of our network)
 - New estimate of loss function
 - Print out the metric value

In [10]:
def random_batch(X, y, batch_size=32):
    idxs = np.random.randint(0, X.shape[0], size=batch_size)
    return X[idxs], y[idxs]


In [11]:
def l1(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true-y_pred))

In [12]:
epochs = 100
batch_size = 32
steps_per_epoch = X_train.shape[0]//batch_size # // means integer division
loss_fn = tf.keras.losses.mean_squared_error

In [13]:
tf.keras.backend.set_floatx('float64') # Remove the warning

In [14]:
model = Sequential([
    Dense(30, activation='relu', input_shape=X_train.shape[1:]), 
    Dense(1)
])

In [15]:
from tensorflow.keras.optimizers import SGD

In [16]:
optimizer = SGD(learning_rate=3e-3)

In [17]:
# Add a validation set
X_train_scl, X_val_scl, y_train, y_val = train_test_split(X_train_scl, y_train, random_state=123)

In [18]:
for epoch in range(epochs):
    for step in range(steps_per_epoch):
        # Get a batch with our sampler
        X_batch, y_batch = random_batch(X_train_scl, y_train, batch_size)
        X_batch = scl.fit_transform(X_batch)
        
        with tf.GradientTape() as tape:
            y_pred = model(X_batch) # Model predictions on the batch
            loss = tf.reduce_mean(loss_fn(y_batch, y_pred)) # Measure loss function on the preds of current batch
            
        grads = tape.gradient(loss, model.variables) # Calculate gradient of the loss function wrt model.variables
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars) # Commit our calculated gradients. Model is modified in-place
    
    # Look at the loss function after an epoch is completed
    y_pred = model(X_train_scl)
    epoch_loss = tf.reduce_mean(loss_fn(y_train, y_pred))
    
    # Same for the validation set
    y_pred = model(X_val_scl)
    epoch_val_loss = tf.reduce_mean(loss_fn(y_val, y_pred))
    print("Epoch: ", epoch, "loss function (train set)", epoch_loss.numpy(), "loss function (val set)", epoch_val_loss.numpy())

Epoch:  0 loss function (train set) 1.5412447442065607 loss function (val set) 2.000838747133028
Epoch:  1 loss function (train set) 1.4615463869813143 loss function (val set) 1.7858828097282864
Epoch:  2 loss function (train set) 1.4286088692106642 loss function (val set) 1.686841785638539
Epoch:  3 loss function (train set) 1.4070622714992398 loss function (val set) 1.6066884683415603
Epoch:  4 loss function (train set) 1.3848199842100393 loss function (val set) 1.5576846423743742
Epoch:  5 loss function (train set) 1.375694851750518 loss function (val set) 1.5218410374022413
Epoch:  6 loss function (train set) 1.370753842635492 loss function (val set) 1.4801169898063036
Epoch:  7 loss function (train set) 1.3640744156365787 loss function (val set) 1.4556224919823384
Epoch:  8 loss function (train set) 1.3583666965187715 loss function (val set) 1.4430871888824424
Epoch:  9 loss function (train set) 1.3550475394305996 loss function (val set) 1.4255395620778724
Epoch:  10 loss function

Epoch:  84 loss function (train set) 1.3377408289629384 loss function (val set) 1.3356372016980604
Epoch:  85 loss function (train set) 1.3382098526627066 loss function (val set) 1.3363436693980943
Epoch:  86 loss function (train set) 1.337841707192107 loss function (val set) 1.335388061949932
Epoch:  87 loss function (train set) 1.3376696340972634 loss function (val set) 1.3352677866716196
Epoch:  88 loss function (train set) 1.3382488833072876 loss function (val set) 1.335623102129067
Epoch:  89 loss function (train set) 1.3380723253938973 loss function (val set) 1.3361457049979957
Epoch:  90 loss function (train set) 1.3386806806168847 loss function (val set) 1.3365712604979472
Epoch:  91 loss function (train set) 1.3379570006450465 loss function (val set) 1.3357212484990644
Epoch:  92 loss function (train set) 1.3381439307931429 loss function (val set) 1.3358866883507523
Epoch:  93 loss function (train set) 1.3375267398813981 loss function (val set) 1.3346656092591602
Epoch:  94 lo